<H1 style="color: cornflowerblue; ">Initialization Libraries</H1>

In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

import os
from PIL import Image

<H1 style="color: cornflowerblue">Connecting CUDA</H1>

In [62]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
device

device(type='cuda')

<H1 style="color: cornflowerblue;">Creating Class Dataset</H1>

In [63]:
class DogsVsCats(Dataset):
    def __init__(self, path_dir1, path_dir2, transform=None):
        self.transform = transform
        self.dog_paths = [(os.path.join(path_dir1, f)) for f in sorted(os.listdir(path_dir1))]
        self.cat_paths = [(os.path.join(path_dir2, f)) for f in sorted(os.listdir(path_dir2))]

    def __len__(self):
        return len(self.dog_paths) + len(self.cat_paths)
    
    def __getitem__(self, i):
        if i < len(self.dog_paths):
            label = 0
            img_path = self.dog_paths[i]
        else:
            label = 1
            img_path = self.cat_paths[i - len(self.dog_paths)]

        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        
        label = torch.tensor(label)

        return img, label

<H1 style="color: cornflowerblue;">Augmentation and transfer data to class Dataset</H1>

In [64]:
transform = transforms.Compose([
    transforms.Resize((300, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [65]:
dogs_path_train = "./data/train/Dog/"
cats_path_train = "./data/train/Cat/"

dogs_path_test = "./data/test/Dog/"
cats_path_test = "./data/test/Cat/"

dogs_vs_cats_train_dataset = DogsVsCats(dogs_path_train, cats_path_train, transform)
dogs_vs_cats_test_dataset = DogsVsCats(dogs_path_test, cats_path_test, transform)

In [66]:
train_loader = DataLoader(dataset=dogs_vs_cats_train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=dogs_vs_cats_test_dataset, batch_size=64, shuffle=False)

<H1 style="color: cornflowerblue;">Connecting and Configurating the pre-trained ResNet18 model</H1>

In [67]:
model_resnet = models.resnet18(pretrained=True)
model_resnet = model_resnet.to(device)

count_classes = 2
model_resnet.fc = nn.Linear(model_resnet.fc.in_features, count_classes).to(device)

model_resnet.fc

c:\Users\ilyae\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ilyae\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Linear(in_features=512, out_features=2, bias=True)

In [80]:
model_resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

<p style="color: tan;">Here I have freezed all layers instead of the last that we have to change for our task.<br>
In front of this code cell, I have written 2 output layers for recognizing Dogs and Cats.  
</p>

In [68]:
for param in model_resnet.parameters():
    param.requires_grad = False

for param in model_resnet.fc.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet.fc.parameters(), lr=0.001)

<p style="color: tan;">Training this ResNet18 model with 10 epochs at a leaning rate of 0.001</p>

In [69]:
epochs = 10

loss_pd = []

for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        prediction = model_resnet(inputs)
        loss = criterion(prediction, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    loss_pd.append(running_loss/len(train_loader))
    print(f"Epoch: [{epoch + 1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

c:\Users\ilyae\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\TiffImagePlugin.py:870: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch: [1/10], Loss: 0.4453
Epoch: [2/10], Loss: 0.2875
Epoch: [3/10], Loss: 0.2371
Epoch: [4/10], Loss: 0.2173
Epoch: [5/10], Loss: 0.2041
Epoch: [6/10], Loss: 0.1904
Epoch: [7/10], Loss: 0.1841
Epoch: [8/10], Loss: 0.1805
Epoch: [9/10], Loss: 0.1745
Epoch: [10/10], Loss: 0.1681


<H1 style="color: cornflowerblue;">Visualization data</H1>

In [79]:
data = []
k = 1
for loss in loss_pd:
    data.append((k, loss))
    k += 1

df = pd.DataFrame(data, columns=['Epochs', 'Loss'])

styles = [
    dict(selector="th", props=[("border", "1px solid black")]),
    dict(selector="td", props=[("border", "1px solid black")]),
]

html = df.style.set_table_styles(styles).to_html()

html = html.replace('<thead>','<thead><tr><th></th>')

HTML(html)

<H1 style="color: cornflowerblue;">Accuracy on test data</H1>

In [ ]:
model_resnet.eval()
model_resnet.cpu()

y_pred = []
y_true = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.cpu(), labels.cpu()
        prediction = model_resnet(images)
        prediction = torch.argmax(prediction, dim=1)
        prediction = prediction.numpy()
        labels = labels.numpy()

        y_pred.append(prediction)
        y_true.append(labels)

y_pred = np.concatenate(y_pred)
y_true = np.concatenate(y_true)

accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {100 * accuracy:.1f}%")

Accuracy: 94.4%
